# Added:

- LSTM layer and linear layer





# Install packages

In [ ]:
pip install gym
pip install yfinance
pip install numpy --upgrade #check if numpy is the most recent version

# Google drive mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pathSP = "/content/drive/MyDrive/Colab Notebooks/Thesis code/data/SP2014-2016.csv"

In [3]:
pathMSFT = "/content/drive/MyDrive/Colab Notebooks/Thesis code/data/MSFT2014-2016.csv"

In [ ]:
pathcorrSP = "/content/drive/MyDrive/Colab Notebooks/Thesis code/data/correlation_test_SP.csv"

In [ ]:
pathcorrMSFT = "/content/drive/MyDrive/Colab Notebooks/Thesis code/data/correlation_test_MSFT.csv"

# Trading_env

In [4]:
import random
from statistics import mean
import warnings
warnings.filterwarnings("ignore")


class TradingSystem_v0:
    def __init__(self, returns_data, k_value, action_dim, initial_balance,mode,stock):
        self.mode = mode  # test or train
        self.initial_balance = initial_balance
        self.balance = self.initial_balance
        self.k = k_value
        self.tickers = stock
        self.curr_position = 0
        self.action_space_dim = action_dim
        self.total_steps = returns_data.shape[0] - self.k
        self.current_step = 0
        self.reward = 0.0
        self.core = float(returns_data.iloc[returns_data.shape[0]-1,3])
        self.dev = float(np.std(returns_data.iloc[:,3]))
        self.trans_cost = 0.0005
        self.slippage_rate = 0.001
        for i in range(0,returns_data.shape[1]):
            col = np.asarray(returns_data.iloc[:,i])
            norm_value = preprocessing.normalize([col]).squeeze()
            for j in range(0,returns_data.shape[0]):
                returns_data.iloc[j,i] = norm_value[j]
        returns_data = returns_data.assign(MA = MA(returns_data.iloc[:,3],10), MACD = MACD(returns_data.iloc[:,3],10,20)
        ,ROC = ROC(returns_data.iloc[:,3],10), SR = SR(returns_data.iloc[:,3],10))
        self.initial_state = returns_data.iloc[0,:]
        self.state = self.initial_state
        self.batch = 20
        self.r_ts = returns_data
        self.is_terminal = False

    # write step function that returns obs(next state), reward, is_done
    def step(self, action):
        self.current_step += 1
        if self.current_step == self.total_steps:
            self.is_terminal = True


        self.pos_change = action - self.action_space_dim//2  # get action(position) of the actor

        self.curr_position = self.curr_position + self.pos_change

        if self.pos_change == 0:

            close_price = self.r_ts.iloc[self.current_step,3]
            close_price_bf = self.r_ts.iloc[self.current_step-1,3]
            self.reward = self.pos_change * (close_price - close_price_bf)


        else:
            close_price = self.r_ts.iloc[self.current_step,3]
            close_price_bf = self.r_ts.iloc[self.current_step-1,3]
            self.reward = self.pos_change * (close_price - close_price_bf)  - abs(self.pos_change)* (self.trans_cost+self.slippage_rate)


        self.state = self.r_ts.iloc[self.current_step+1,:]

        return self.state, self.reward, self.is_terminal

    def reset(self):
        self.r_ts = self.r_ts
        self.total_steps = len(self.r_ts) - self.k
        self.balance = self.initial_balance
        self.current_step = 0
        self.initial_state = self.r_ts.iloc[0,:]
        self.state = self.initial_state
        self.reward = 0.0
        self.is_terminal = False
        return self.state



# SDAE

In [5]:
import os
import sys
import json
import argparse
from collections import defaultdict
import tensorflow as tf
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.functional import F
from datetime import tzinfo, timedelta, datetime

class SDAE(nn.Module):
    def __init__(self, input_dim):
        super(SDAE, self).__init__()
        self.num_layers = 4

        # Encoder layers
        self.enc_fc_1 = nn.Linear(input_dim, 10)
        self.enc_fc_2 = nn.Linear(10, 16)
        self.encoder_object = nn.Sequential(*[self.enc_fc_1, nn.ReLU(), self.enc_fc_2])

        # Decoder layers
        self.dec_fc_1 = nn.Linear(16, 10)
        self.dec_fc_2 = nn.Linear(10, input_dim)
        self.decoder_object = nn.Sequential(*[self.dec_fc_1, nn.ReLU(), self.dec_fc_2])

    def forward(self, input, training=True):
        if training:
            encoded_output = self.encoder_object(input)
            decoded_output = self.decoder_object(encoded_output)
            return decoded_output
        else:
            with torch.no_grad():
                encoded_output = self.encoder_object(input)
                return encoded_output

    def freeze_all_but(self, layer_index):
        if layer_index >= self.num_layers:
            print("Layer index error")
            exit()
        layer_count = 0
        for name, param in self.encoder_object.named_parameters():
            if layer_index != layer_count:
                param.requires_grad = False
            if 'bias' in name:
                layer_count += 1
        layer_count = self.num_layers - 1
        for name, param in self.decoder_object.named_parameters():
            if layer_index != layer_count:
                param.requires_grad = False
            if 'bias' in name:
                layer_count -= 1

    def unfreeze_all(self):
        for name, param in self.encoder_object.named_parameters():
            param.requires_grad = True
        for name, param in self.decoder_object.named_parameters():
            param.requires_grad = True

# dqn

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import math
import numpy as np


class MLP(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(state_dim, 64)  # input layer
        self.fc2 = nn.Linear(64, hidden_dim)  # hidden layer 1
        self.fc3 = nn.Linear(hidden_dim, 128)  # hidden layer 2
        self.lstm_cells = nn.LSTMCell(128, 128) # lstm layer (returns hidden state and cell state)
        self.fc4 = nn.Linear(128, action_dim) # connects lstm layer to linear layer

    def forward(self, x):
        # activation function
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = torch.nn.functional.relu(self.fc3(x))
        hx,cx = self.lstm_cells(x)
        # x = hx.clone().detach()
        # x = torch.tensor(hx) (gives warning)


        return self.fc4(hx)


class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity  # capacity of buffer
        self.buffer = []  # replay buffer
        self.position = 0

    def push(self, state, action, reward, next_state, done):
        ''' replay buffer is a queue (LIFO)
        '''
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = zip(*batch)
        return state, action, reward, next_state, done

    def __len__(self):
        return len(self.buffer)


class DQN:
    def __init__(self, state_dim, action_dim, cfg):

        self.action_dim = action_dim
        self.device = cfg.device  # cpu or gpu
        self.gamma = cfg.gamma  # discount factor
        self.frame_idx = 0  # attenuation
        self.epsilon = lambda frame_idx: cfg.epsilon_end + \
                                         (cfg.epsilon_start - cfg.epsilon_end) * \
                                         math.exp(-1. * frame_idx / cfg.epsilon_decay)
        self.batch_size = cfg.batch_size
        self.policy_net = MLP(state_dim, action_dim, hidden_dim=cfg.hidden_dim).to(self.device)
        self.target_net = MLP(state_dim, action_dim, hidden_dim=cfg.hidden_dim).to(self.device)
        for target_param, param in zip(self.target_net.parameters(),
                                       self.policy_net.parameters()):  # copy parameters to target net
            target_param.data.copy_(param.data)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=cfg.lr) # optimizer
        self.memory = ReplayBuffer(cfg.memory_capacity)  # experience replay

    def choose_action(self, state):
        self.frame_idx += 1
        if random.random() > self.epsilon(self.frame_idx):
            with torch.no_grad():
                input_state = torch.tensor([state], device=self.device, dtype=torch.float32)   # received observation
                sdae = SDAE(input_state.shape[1])
                output_state = sdae.forward(input_state)     # denoised state (using SDAE)
                k = torch.stack((input_state.squeeze(),output_state.squeeze()))
                cor = torch.corrcoef(k)[0,1].item()
                q_values = self.policy_net(output_state)
                action = q_values.max(1)[1].item()  # choose the action with maximum q-value

        else:
            input_state = torch.tensor([state], device=self.device, dtype=torch.float32)
            sdae = SDAE(input_state.shape[1])
            output_state = sdae.forward(input_state)
            action = random.randrange(self.action_dim) # pick random action
            k = torch.stack((input_state.squeeze(),output_state.squeeze()))
            cor = torch.corrcoef(k)[0,1].item()
        return action, cor

    def update(self):
        if len(self.memory) < self.batch_size:
            return
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = self.memory.sample(
            self.batch_size)
        # transfer to tensor
        state_batch = torch.tensor(state_batch, device=self.device, dtype=torch.float)
        action_batch = torch.tensor(action_batch, device=self.device).unsqueeze(1)
        reward_batch = torch.tensor(reward_batch, device=self.device, dtype=torch.float)
        next_state_batch = torch.tensor(next_state_batch, device=self.device, dtype=torch.float)
        done_batch = torch.tensor(np.float32(done_batch), device=self.device)
        q_values = self.policy_net(state_batch).gather(dim=1, index=action_batch)
        next_q_values = self.target_net(next_state_batch).max(1)[0].detach()
        # calculate the expected q-value, for final state, done_batch[0]=1 and the corresponding
        # expected_q_value equals to reward
        expected_q_values = reward_batch + self.gamma * next_q_values * (1 - done_batch)
        loss = nn.MSELoss()(q_values, expected_q_values.unsqueeze(1))
        # update the network
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():  # avoid gradient explosion by using clip
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

    def save(self, path):
        torch.save(self.target_net.state_dict(), path + 'dqn_checkpoint.pth')

    def load(self, path):
        self.target_net.load_state_dict(torch.load(path + 'dqn_checkpoint.pth'))
        for target_param, param in zip(self.target_net.parameters(), self.policy_net.parameters()):
            param.data.copy_(target_param.data)

# Main

In [16]:
import sys
import os
import os.path


curr_path = os.path.abspath('')

import gym
import torch
import numpy as np
import random
import yfinance as yf
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt

curr_time = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
random.seed(11)



class Config:
    '''
    hyperparameters
    '''

    def __init__(self):
        ################################## env hyperparameters ###################################
        self.algo_name = 'DQN' # algorithmic name
        self.env_name = 'TradingSystem_v0' # environment name
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")  # examine GPU
        self.seed = 11 # random seed
        self.train_eps = 100 # number of training episode
        self.state_space_dim = 8 # state space size
        self.initial_balance = 1000000
        self.action_space_dim = 3 # action space size (only odd number can be applied)
        ################################################################################

        ################################## algo hyperparameters ###################################
        self.gamma = 0.95  # discount factor
        self.epsilon_start = 0.90  # start epsilon of e-greedy policy
        self.epsilon_end = 0.01  # end epsilon of e-greedy policy
        self.epsilon_decay = 500  # attenuation rate of epsilon in e-greedy policy
        self.lr = 0.0001  # learning rate
        self.memory_capacity = 1000  # capacity of experience replay
        self.batch_size = 64  # size of mini-batch SGD
        self.target_update = 4  # update frequency of target network
        self.hidden_dim = 128  # dimension of hidden layer
        ################################################################################

        ################################# save path ##############################
        self.result_path = curr_path + "/outputs/" + self.env_name + \
                           '/' + curr_time + '/results/'
        self.model_path = curr_path + "/outputs/" + self.env_name + \
                          '/' + curr_time + '/models/'
        self.save = True  # whether to save the image
        ################################################################################


def env_agent_config(data, cfg, mode,stock):
    ''' create environment and agent
    '''
    env = TradingSystem_v0(data, cfg.state_space_dim, cfg.action_space_dim, cfg.initial_balance, mode, stock)
    agent = DQN(cfg.state_space_dim, cfg.action_space_dim, cfg)

    if cfg.seed != 0:  # set random seeds
        torch.manual_seed(cfg.seed)
        np.random.seed(cfg.seed)
    return env, agent


def train(cfg, env, agent):
    ''' training
    '''
    print('Start Training!')
    print(f'Environment：{cfg.env_name}, Algorithm：{cfg.algo_name}, Device：{cfg.device}, Stock: {env.tickers}')
    rewards = []  # record total rewards
    ma_rewards = []  # record moving average total rewards
    stocks = env.tickers
    ep_reward = 0
    state = env.reset()
    for i_ep in range(cfg.train_eps):
        ep_reward = 0
        state = env.reset()
        while True:
            action, cor = agent.choose_action(state)
            next_state, reward, done = env.step(action)
            agent.memory.push(state, action, reward, next_state, done)  # save transition
            state = next_state
            agent.update()
            ep_reward +=  reward
            if done:
                break
        if (i_ep + 1) % cfg.target_update == 0:  # update target network
            agent.target_net.load_state_dict(agent.policy_net.state_dict())
        sum_rewards = env.core * env.batch * (1+ep_reward) + env.dev * ep_reward
        rewards.append(sum_rewards)
        if ma_rewards:
            ma_rewards.append(0.9 * ma_rewards[-1] + 0.1 * sum_rewards)
        else:
            ma_rewards.append(sum_rewards)
        if (i_ep + 1) % 10 == 0:
            print('Episode：{}/{}, Reward：{}'.format(i_ep + 1, cfg.train_eps, sum_rewards))
    print('Finish Training!')
    return rewards, ma_rewards


def test(cfg, env, agent):
    print('Start Testing!')
    print(f'Environment：{cfg.env_name}, Algorithm：{cfg.algo_name}, Device：{cfg.device},Stock: {env.tickers}')
    ############# Test does not use e-greedy policy, so we set epsilon to 0 ###############
    cfg.epsilon_start = 0.0
    cfg.epsilon_end = 0.0
    ################################################################################
    stocks = env.tickers
    returns = []
    correlation = []
    ep_reward = 0
    state = env.reset()
    while True:
        action,cor = agent.choose_action(state)
        next_state, reward, done = env.step(action)
        state = next_state
        ep_reward += reward
        correlation.append(cor)
        if done:
            break
    sum_rewards = ep_reward * env.dev + env.core
    returns.append(sum_rewards)
    print(f"Stock：{stocks}，Return：{sum_rewards:.1f}")
    print('Finish Testing!')
    return stocks,sum_rewards,correlation


# Indicators

In [10]:
def MA(sorted_data, period):
	ma_list = []
	for i in range(len(sorted_data)):
		if(i < period - 1):
			ma_list.append(sorted_data[i])
			#ma_list.append(None)
		else:
			ma_sum = 0.0
			for j in range(i, i - period, -1):
				ma_sum += sorted_data[j]
			curr_ma = ma_sum / period
			ma_list.append(curr_ma)
	return ma_list

def EMA(data, period):
    sorted_data = np.array([data[i] for i in sorted(data.keys())])
    sma = np.mean(sorted_data[:period])
    ema_list = []
    smooth_weighting = 2.0 / (period + 1)
    for i in range(len(sorted_data)):
        if i < period - 1:
            ema_list.append(sma)
        else:
            new_ema = (sorted_data[i] - ema_list[-1]) * smooth_weighting + ema_list[-1]
            ema_list.append(new_ema)
    assert len(ema_list) == len(sorted_data)
    return np.array(ema_list)

def MACD(sorted_data, period_1, period_2):
	period_1_ema_list = EMA(sorted_data, period_1)
	period_2_ema_list = EMA(sorted_data, period_2)
	assert len(period_1_ema_list) == len(period_2_ema_list)
	macd_list = period_1_ema_list - period_2_ema_list
	return macd_list


def ROC(data, period):
    roc_list = []
    for i in range(len(data)):
        if i == 0:
            roc_list.append(0.0)
        elif i < period:
            curr_roc = (data[i] - data[0]) / data[0]
            roc_list.append(curr_roc)
        else:
            curr_roc = (data[i] - data[i - period]) / data[i - period]
            roc_list.append(curr_roc)
    return roc_list

def SR(sorted_data, period, risk_free_rate=0.007):
    # trading hours per day
    risk_free_rate = risk_free_rate / (6.5 * 60)
    sr_list = []
    for i in range(len(sorted_data)):
        if i < period:
            sr_list.append(0.0)
        else:
            expected_return_list = []
            for j in range(i, i - period, -1):
                perc_change = (sorted_data[j] - sorted_data[j - 1]) / sorted_data[j - 1]
                expected_return_list.append(perc_change)
            expected_return_list = np.array(expected_return_list)
            std = np.std(expected_return_list)
            if std != 0.0:
                sr_list.append((np.mean(expected_return_list) - risk_free_rate) / std)
            else:
                sr_list.append(0.0)
    return sr_list


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Load Dataset

In [9]:
import pandas as pd
from sklearn import preprocessing
import math

### S&P 500 ###
# SP = pd.read_csv(r'/Users/gilseyeon/Downloads/data/SPXUSD/SP2012.csv')

SP = pd.read_csv(pathSP)
oo = range(0,len(SP),30)
returns_SP = SP.iloc[oo,].reset_index(drop=True)

returns_SP = returns_SP.iloc[:,2:6] # remove time stamps

#splitting train and test set
splitter = math.ceil(returns_SP.shape[0]*0.9)
SP_train = returns_SP.iloc[0:splitter,:]
SP_test = returns_SP.iloc[splitter+1:,:].reset_index(drop=True)


### Microsoft ###
MSFT = pd.read_csv(pathMSFT)
dd = range(0,len(MSFT),6)
returns_MSFT = MSFT.iloc[dd,].reset_index(drop=True)
returns_MSFT = returns_MSFT.iloc[:,1:5] # remove time stamps

#splitting train and test set
splitter = math.ceil(returns_MSFT.shape[0]*0.9)
MSFT_train = returns_MSFT.iloc[0:splitter,:]
MSFT_test = returns_MSFT.iloc[splitter+1:,:].reset_index(drop=True)



# Train / Test

In [ ]:
#Training SP500
cfg = Config()
env, agent = env_agent_config(SP_train, cfg, 'train','SP500')
rewards_train_SP, ma_rewards_train_SP = train(cfg, env, agent)

Start Training!
Environment：TradingSystem_v0, Algorithm：DQN, Device：cpu, Stock: SP500
Episode：10/100, Reward：29756.582744172407
Episode：20/100, Reward：34149.27518358311
Episode：30/100, Reward：35411.574554890074
Episode：40/100, Reward：35407.284238092245
Episode：50/100, Reward：33821.865815207275
Episode：60/100, Reward：34265.25656620571
Episode：70/100, Reward：35094.19199431144
Episode：80/100, Reward：36794.19383142421
Episode：90/100, Reward：36928.97547866655
Episode：100/100, Reward：36049.42484767726
Finish Training!


In [ ]:
rewards_train_SP

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[-159919.26372262044,
 -111319.55145007331,
 -75752.94503514578,
 -39130.809659565944,
 -12324.762133000098,
 3697.3731945779364,
 20512.526767792802,
 23908.818594901102,
 25541.91256062946,
 29756.582744172407,
 29957.646481481424,
 29961.21371117852,
 28685.175684957117,
 31267.934803321994,
 32545.152907691834,
 30851.225002225452,
 31861.801083997663,
 32431.14796314464,
 34001.09232207336,
 34149.27518358311,
 33307.90265734432,
 32488.415567355183,
 32194.219999807552,
 29868.755868064352,
 35279.82987347092,
 34142.04431257552,
 35722.56888196255,
 34715.39613014457,
 35972.48927432607,
 35411.574554890074,
 35355.27106681388,
 34848.442368345095,
 33436.36934390671,
 33755.05346126671,
 34644.293459406166,
 35791.9130048719,
 33568.137164113075,
 34081.06486124774,
 34897.804220253995,
 35407.284238092245,
 35733.48846130014,
 35645.995816277864,
 36040.41036182114,
 33385.175671342535,
 35609.69773798934,
 35858.146889131865,
 36618.67917167536,
 36039.15701084649,
 34384.756

In [ ]:
ma_rewards_train_SP

[-159919.26372262044,
 -155059.29249536572,
 -147128.65774934372,
 -136328.87294036595,
 -123928.46185962936,
 -111165.87835420863,
 -97998.0378420085,
 -85807.35219831755,
 -74672.42572242285,
 -64229.52487576333,
 -54810.80774003886,
 -46333.60559491712,
 -38831.7274669297,
 -31821.761239904532,
 -25385.069825144896,
 -19761.440342407863,
 -14599.116199767312,
 -9896.089783476116,
 -5506.371572921169,
 -1540.806897270741,
 1944.064058190765,
 4998.499209107207,
 7718.071288177242,
 9933.139746165954,
 12467.808758896452,
 14635.23231426436,
 16743.96597103418,
 18541.10898694522,
 20284.247015683304,
 21796.979769603982,
 23152.808899324973,
 24322.372246226987,
 25233.77195599496,
 26085.900106522135,
 26941.73944181054,
 27826.756798116676,
 28400.894834716317,
 28968.91183736946,
 29561.801075657913,
 30146.34939190135,
 30705.06329884123,
 31199.156550584892,
 31683.281931708516,
 31853.47130567192,
 32229.09394890366,
 32591.999242926482,
 32994.667235801375,
 33299.11621330589,

In [ ]:
# Testing SP500
env, agent = env_agent_config(SP_test, cfg, 'test',"SP500")
stocks_test_SP, rewards_test_SP, correlation_test_SP = test(cfg, env, agent)

Start Testing!
Environment：TradingSystem_v0, Algorithm：DQN, Device：cpu,Stock: SP500
Stock：SP500，Return：2137.4
Finish Testing!


In [ ]:
rewards_test_SP = 2137.386746022535

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2137.386746022535

In [ ]:
correlation_test_SP = pd.DataFrame(correlation_test_SP).to_csv('/content/drive/My Drive/correlation_test_SP.csv')

In [ ]:
# Training Microsoft
cfg = Config()
env, agent = env_agent_config(MSFT_train, cfg, 'train','MSFT')
rewards_train_MSFT, ma_rewards_train_MSFT = train(cfg, env, agent)

Start Training!
Environment：TradingSystem_v0, Algorithm：DQN, Device：cpu, Stock: MSFT
Episode：10/100, Reward：959.9293519126438
Episode：20/100, Reward：897.2879098321079
Episode：30/100, Reward：908.4821707613372
Episode：40/100, Reward：971.73837924582
Episode：50/100, Reward：943.3148046943527
Episode：60/100, Reward：961.7204148844937
Episode：70/100, Reward：976.7511805015234
Episode：80/100, Reward：967.8779757556414
Episode：90/100, Reward：981.9977606428424
Episode：100/100, Reward：1011.0210801639878
Finish Training!


In [ ]:
rewards_train_MSFT

[-4134.2044817276565,
 -2463.6940359760524,
 -960.3834196956681,
 380.8042370256189,
 805.9308171815887,
 861.5548298088413,
 834.4596308514432,
 909.5001714142063,
 926.9579832746917,
 959.9293519126438,
 850.9567736954604,
 860.5269167252496,
 841.8569132825774,
 924.7233068447848,
 882.6642404499964,
 909.5985876906674,
 894.1043837541355,
 952.6672415912049,
 878.5702940844576,
 897.2879098321079,
 875.0785789241758,
 888.6836039890852,
 881.7926760326435,
 909.5896193009659,
 971.7573244326775,
 931.6002269740932,
 944.2598996338111,
 940.9693107218429,
 961.0091014304912,
 908.4821707613372,
 930.8074873821712,
 965.0394323044873,
 950.589533436552,
 955.6508669829365,
 950.0669183095617,
 950.8396613962165,
 970.0393388180601,
 945.745779890078,
 952.8064018199221,
 971.73837924582,
 939.9491456655608,
 898.2010120543936,
 973.8046516308298,
 932.062034677286,
 957.9370794704101,
 931.5474035878611,
 943.1755183326093,
 913.5023312459105,
 902.676546691641,
 943.3148046943527,
 

In [ ]:
ma_rewards_train_MSFT

[-4134.2044817276565,
 -3967.153437152496,
 -3666.476435406813,
 -3261.74836816357,
 -2854.980449629054,
 -2483.3269216852646,
 -2151.5482664315937,
 -1845.4434226470137,
 -1568.2032820548432,
 -1315.3900186580945,
 -1098.7553394227389,
 -902.8271138079401,
 -728.3587110988883,
 -563.050509304521,
 -418.4790343290693,
 -285.6712721270956,
 -167.6937065389725,
 -55.65761172595475,
 37.76517885508649,
 123.71745195278862,
 198.85356464992734,
 267.83656858384313,
 329.2321793287232,
 387.26792332594744,
 445.71686343662043,
 494.30519979036774,
 539.3006697747121,
 579.4675338694252,
 617.6216906255318,
 646.7077386391122,
 675.1177135134182,
 704.109885392525,
 728.7578501969277,
 751.4471518755286,
 771.3091285189319,
 789.2621818066604,
 807.3398975078003,
 821.1804857460281,
 834.3430773534176,
 848.0826075426579,
 857.2692613549482,
 861.3624364248928,
 872.6066579454866,
 878.5521956186666,
 886.4906840038409,
 890.9963559622429,
 896.2142721992795,
 897.9430781039428,
 898.4164249

In [ ]:
rewards_train_MSFT = pd.DataFrame(rewards_train_MSFT).to_csv('/content/drive/My Drive/rewards_train_MSFT.csv')

In [ ]:
ma_rewards_train_MSFT = pd.DataFrame(ma_rewards_train_MSFT).to_csv('/content/drive/My Drive/ma_rewards_train_MSFT.csv')

In [ ]:
import numpy as np
import pandas as pd
correlation_train_MSFT = pd.DataFrame(correlation_train_MSFT).to_csv('/content/drive/My Drive/correlation_train_MSFT.csv')

In [12]:
# Testing Microsoft
cfg = Config()
env, agent = env_agent_config(MSFT_test, cfg, 'test',"MSFT")
stocks_test_MSFT, rewards_test_MSFT, correlation_test_MSFT = test(cfg, env, agent)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Start Testing!
Environment：TradingSystem_v0, Algorithm：DQN, Device：cpu,Stock: MSFT
Stock：MSFT，Return：59.6
Finish Testing!


In [15]:
rewards_test_MSFT

59.6021745894492

In [ ]:
save_correlation_test_MSFT = pd.DataFrame(correlation_test_MSFT).to_csv('/content/drive/My Drive/correlation_test_MSFT.csv')

# Save results